# Начальная инициализация

Подключение библиотек

In [1]:
import torch
import sklearn.model_selection
import torchmetrics
import IPython.display
from tqdm import tqdm
import lightning
import pandas as pd
import numpy as np

Воспроизводимость результатов

In [2]:
random_state = 42

lightning.seed_everything(random_state, workers=True)

Global seed set to 42


42

# Задание

Сетка перебираемых гиперпараметров

In [3]:
param_grid = {
    "layers_count": [2, 3, 4],
    "learning_rate": [0.01, 0.001, 0.0001],
    "layers_type": ['different', 'equal'],
    "batch_size": [8, 64],
    "optimizer": [torch.optim.SGD],
    'epochs': [5, 10],
    'activation_function': [torch.nn.functional.relu],
}

params_list = sklearn.model_selection.ParameterGrid(param_grid)
len(params_list)

72

Класс для работы с данными

In [4]:
class NetData(lightning.LightningDataModule):
    def __init__(self, train_features=None, test_features=None, train_targets=None, test_targets=None, batch_size=None, random_state=None):
        super().__init__()
        
        self.batch_size = batch_size
        self.X_train, self.X_val, self.y_train, self.y_val = sklearn.model_selection.train_test_split(train_features, train_targets, random_state=random_state)
        self.X_test, self.y_test = test_features, test_targets

    def setup(self, stage=None):
        features_train = torch.tensor(self.X_train, dtype=torch.float32)
        targets_train = torch.tensor(self.y_train, dtype=torch.int32)
    
        features_val = torch.tensor(self.X_val, dtype=torch.float32)
        targets_val = torch.tensor(self.y_val, dtype=torch.int32)
    
        features_test = torch.tensor(self.X_test, dtype=torch.float32)
        targets_test = torch.tensor(self.y_test, dtype=torch.int32)
    
        self.trainset = torch.utils.data.TensorDataset(features_train, targets_train)
        self.valset = torch.utils.data.TensorDataset(features_val, targets_val)
        self.testset = torch.utils.data.TensorDataset(features_test, targets_test)
        
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.trainset, batch_size=self.batch_size)
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valset, batch_size=self.batch_size)
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.testset, batch_size=self.batch_size)

Загрузка данных

In [5]:
X_train = pd.read_csv('manual_features/X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('manual_features/y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('manual_features/X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('manual_features/y_test_Final.csv', names=['class']).values.tolist()])

Нейронная сеть

In [6]:
#Архитектура нейронной сети
class Net(lightning.LightningModule):
    def __init__(self,
                 learning_rate=None,
                 layers_count=None,
                 layers_type=None,
                 optimizer=None,
                 activation=None
                 ):
        
        super().__init__()  #вх. #вых.
        self.layers = torch.nn.ModuleList()
        input_dim = 2**(2 + layers_count)
        self.layers.append(torch.nn.Linear(20, input_dim))
        for _ in range(layers_count):
            if layers_type == 'equal':
                self.layers.append( torch.nn.Linear(input_dim, input_dim) )
            elif layers_type == "different":
                self.layers.append( torch.nn.Linear(input_dim, input_dim // 2) )
                input_dim //= 2
        self.layers.append(torch.nn.Linear(input_dim, 2))

        self.activation = activation

        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.f1 = torchmetrics.classification.F1Score(task='multiclass', num_classes=2, multidim_average='global', average='weighted')

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)
        return torch.nn.functional.log_softmax(x, dim=-1)
    
    def configure_optimizers(self):
        optimizer = self.optimizer(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y.long())
        self.log('loss', loss, on_epoch=True)
        self.log("train_f1", self.f1(y_hat, y), on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        self.log("val_f1", self.f1(pred, y), on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        self.log("test_f1", self.f1(pred, y), on_epoch=True)

In [7]:
total = []

for params in tqdm(params_list): 

    learningRate = params['learning_rate']
    optimizer_type = params['optimizer']
    layers_count = params['layers_count']
    batch_size = params['batch_size']
    epochs = params['epochs']
    activation_function = params['activation_function']
    layers_type = params['layers_type']


    net = Net(learning_rate=learningRate,
          layers_count=layers_count,
          layers_type=layers_type,
          optimizer=optimizer_type,
          activation=activation_function
        )
    
    dm = NetData(
        train_features=X_train,
        test_features=X_test,
        train_targets=y_train,
        test_targets=y_test,
        batch_size=batch_size,
        random_state=random_state
    )

    # CPU is better for FCNN than GPU
    trainer = lightning.Trainer(logger=False, max_epochs=epochs, enable_progress_bar=True, deterministic=True, inference_mode=True, enable_checkpointing=False, accelerator='cpu')

    trainer.fit(net, datamodule=dm)
    f1_train = trainer.logged_metrics['train_f1_epoch'].item()
    f1_val = trainer.logged_metrics['val_f1'].item()
    f1_test = trainer.test(net, datamodule=dm)[-1]['test_f1']    
    
    
    total.append({
        'Layers type': layers_type,
        'Optimizer': optimizer_type.__name__,
        'Batch': batch_size,
        'Count of layers': layers_count,
        'Epochs': epochs,
        'Learning rate': learningRate,
        'Activation function': activation_function.__name__,
        'F1-train': round(f1_train, 4),
        'F1-val': round(f1_val, 4),
        'F1-test': round(f1_test, 4)
    })
    
    IPython.display.clear_output(wait=True) 

100%|██████████| 72/72 [1:43:43<00:00, 86.44s/it]


# Сводная таблица

In [8]:
pd.set_option('display.max_rows', None)
summary = pd.DataFrame.from_dict(total)
summary_sort = summary.sort_values(by='F1-val', ascending=False)
summary_sort

,Layers type,Optimizer,Batch,Count of layers,Epochs,Learning rate,Activation function,F1-train,F1-val,F1-test
54,different,SGD,64,2,10,0.0100,relu,0.6071,0.6126,0.6784
57,equal,SGD,64,2,10,0.0100,relu,0.6056,0.6105,0.7200
12,different,SGD,8,4,5,0.0100,relu,0.5868,0.6088,0.6486
60,different,SGD,64,3,10,0.0100,relu,0.5995,0.6081,0.6784
0,different,SGD,8,2,5,0.0100,relu,0.6053,0.6071,0.7133
24,different,SGD,8,3,10,0.0100,relu,0.6083,0.6048,0.6562
33,equal,SGD,8,4,10,0.0100,relu,0.6095,0.6042,0.6035
15,equal,SGD,8,4,5,0.0100,relu,0.6027,0.6032,0.6427
6,different,SGD,8,3,5,0.0100,relu,0.6025,0.6032,0.6427
18,different,SGD,8,2,10,0.0100,relu,0.6084,0.6021,0.6562
